In [1]:
from IPython.core.display import HTML
with open('style.css', 'r') as file:
    css = file.read()
HTML(css)

# Symbolic Differentiation

In this notebook our goal is to implement <em style="color:blue">symbolic differentiation</em>.  Concretely, we want do implement a function `diff` that takes two arguments:
  - The first argument <tt>expr</tt> represents an *arithmetic expression*.
    Here an arithmetic expression is any string that is build from variable and numbers by application
    of any of the operator symbols "`+`", "`-`", "`*`", "`/`", and "`**`".
    The operator "<tt>**</tt>" represents exponentiation, i.e. an expression of the form 
    $a \texttt{**} b$ is interpreted as $a^b$.          
    Furthermore, if $e$ is an expression, then both $\exp(e)$ and $\ln(e)$ are expressions too.
  - The second argument <tt>var</tt> is the name of a variable.</li>

The function call <tt>diff(expr, var)</tt> will then compute the derivative of <tt>expr</tt> with respect to the variable <tt>var</tt>.  For example, the function call <tt>diff("x * exp(x)", "x")</tt> will compute the output
"<tt>1 * exp(x) + x * exp(x)</tt>" because we have:
$$ \frac{\mathrm{d}\;}{\mathrm{d}x} \bigl( x \cdot \mathrm{e}^x \bigr) = 1 \cdot \mathrm{e}^x + x \cdot \mathrm{e}^x $$

It would be very tedious to **represent** arithmetic expressions as strings.  Instead, we will represent arithmetic expressions as *nested tuples*.  For example, the arithmetic expression "<tt>x\*exp(x)</tt>" is represented as the nested tuple <tt>("\*", "x", ("exp", "x"))</tt>.  In order to be able to convert string into nested tuples, we need a parser.  Therefore we import the local file "<tt>exprParser.py</tt>".

In [ ]:
import exprParser as ep

This module provides the class <tt>ExprParser</tt>.  Given a string $s$, the call <tt>ExprParser($s$)</tt> creates a parser for the string $s$.  The method <tt>parse</tt> will then return a *nested tuple* representation of the string $s$.  The following example clarifies this:

In [ ]:
ep.ExprParser('x*exp(x)').parse()

Now we are ready to implement the function <tt>diff($e$)</tt>, where $e$ is a *nested tuple* and $x$ is a variable.

In [ ]:
def diff(e):
    "differentiate the expressions e with respect to the variable x"
    if e[0] == '+':
        f , g  = e[1:]
        fs, gs = diff(f), diff(g)
        return ('+', fs, gs)
    if e[0] == '-':
        f , g  = e[1:]
        fs, gs = diff(f), diff(g)
        return ('-', fs, gs)
    if e[0] == '*':
        f , g  = e[1:]
        fs, gs = diff(f), diff(g)
        return ('+', ('*', fs, g), ('*', f, gs))
    if e[0] == '/':
        f , g  = e[1:]
        fs, gs = diff(f), diff(g)
        return ('/', ('-', ('*', fs, g), ('*', f, gs)), ('*', g, g))
    if e[0] == '**':
        f , g  = e[1:]
        return diff(('exp', ('*', g, ('ln', f))))
    if e[0] == 'ln':
        f  = e[1]
        fs = diff(f) 
        return ('/', fs, f)
    if e[0] == 'exp':
        f  = e[1]
        fs = diff(f) 
        return ('*', fs, e)
    if e == 'x':
        return '1'
    return 0

Let us inspect the details of this function:
<ol>
<li>  The lines 3 - 6 implement the rule: 
      $$\frac{\mathrm{d}\;}{\mathrm{d}x}\bigl(f(x) + g(x)\bigr) = \frac{\mathrm{d}\;}{\mathrm{d}x} f(x) + \frac{\mathrm{d}\;}{\mathrm{d}x} g(x)$$
      </li>
<li>  Line 7 - 10 implement the rule:
      $$\frac{\mathrm{d}\;}{\mathrm{d}x}\bigl(f(x) - g(x)\bigr) = \frac{\mathrm{d}\;}{\mathrm{d}x} f(x) - \frac{\mathrm{d}\;}{\mathrm{d}x} g(x)$$
      </li>
<li>  Line 11 - 14 deals with the case where <tt>e</tt> is a product.  The 
      <a href="https://en.wikipedia.org/wiki/Product_rule">product rule</a> is      
      $$ \frac{\mathrm{d}\;}{\mathrm{d}x}\bigl(f(x) \cdot g(x)\bigr) = \left(\frac{\mathrm{d}\;}{\mathrm{d}x} f(x)\right)\cdot g(x) + f(x) \cdot \left(\frac{\mathrm{d}\;}{\mathrm{d}x} g(x)\right)
      $$
      </li>
<li>  Line 15 - 17 deals with the case where <tt>e</tt> is a quotient.  The
      <a href="https://en.wikipedia.org/wiki/Quotient_rule">quotient rule</a> is
      $$ \frac{\mathrm{d}\;}{\mathrm{d}x}\left(\frac{f(x)}{g(x)}\right) = 
         \frac{\displaystyle\left(\frac{\mathrm{d}\;}{\mathrm{d}x} f(x)\right)\cdot g(x) - 
         f(x) \cdot \left(\frac{\mathrm{d}\;}{\mathrm{d}x} g(x)\right)}{g(x) \cdot g(x)}
      $$
      </li>
<li>  Line 19 - 21 deals with the case where <tt>e</tt> is a power.  Now in order to take the derivative of an
      expression of the form
      $$  f(x)^{g(x)} $$
      we first need to rewrite this expression using the following trick:
      $$ f(x)^{g(x)} = \exp\bigl(\ln\bigl(f(x)^{g(x)}\bigr)\bigr) = \exp\bigl(g(x) \cdot \ln(f(x))\bigr) $$
      Then, we can recursively call <tt>diff</tt> for this expression.  This works, because the function
      <tt>diff</tt> can deal with both the exponential function $x \mapsto \exp(x)$ and with the natural
      logarithm $x \mapsto \ln(x)$.  This rewriting is done in line 21.
      </li>
<li>  Line 22-25 deals with the case where <tt>e</tt> has the form 
      $$\ln\bigl(f(x)\bigr)$$  
      In order to take the derivative of this expression, we first need to know the derivative of the natural
      logarithm.  This derivative is given as     
      $$ \frac{\mathrm{d}\;}{\mathrm{d}x} \ln(x) = \frac{1}{x}$$
      Then, using the <a href="https://en.wikipedia.org/wiki/Chain_rule">chain rule</a> we have that
      $$ \frac{\mathrm{d}\;}{\mathrm{d}x} \ln\bigl(f(x)\bigr) = \frac{\frac{\mathrm{d}\;}{\mathrm{d}x} f(x)}{f(x)}$$
      </li>
<li>  Line 26 - 29 deals with the case where <tt>e</tt> has the form $\exp\bigl(f(x)\bigr)$.  
      In order to take the derivative of this expression, we first need to know the derivative of the 
      <a href="https://en.wikipedia.org/wiki/Exponential_function">exponential function</a>.  
      This derivative is given as 
      $$ \frac{\mathrm{d}\;}{\mathrm{d}x} \exp(x) = \exp(x)$$    
      Then, using the <a href="https://en.wikipedia.org/wiki/Chain_rule">chain rule</a> we have that
      $$\frac{\mathrm{d}\;}{\mathrm{d}x} \exp\bigl(f(x)\bigr) = \left(\frac{\mathrm{d}\;}{\mathrm{d}x} f(x)\right) \cdot \exp\bigl(f(x)\bigr)
      $$
      </li>
<li>  Line 30-31 deals with the case where <tt>e</tt> is a variable and happens to be the same variable as
      <tt>x</tt>.  This is checked using the condition    
      <tt>e == x</tt>.  As we have
      $$\frac{\mathrm{d}x}{\mathrm{d}x} = 1,$$
      the function <tt>diff</tt> returns <tt>1</tt> in this case.
      </li>
<li>  Otherwise, the expression is assumed to be a constant and hence we return 0.
      </li>
</ol>


Let us implement a small function <tt>test</tt> that takes a string $s$, parses this string as an arithmetic expression and then takes the derivative of this expression with respect to the variable "<tt>x</tt>".

In [ ]:
def test(s):
    t = ep.ExprParser(s).parse()
    d = diff(t)
    print(f"d/dx {s} = {ep.toString(d)}")

In [ ]:
test("x ** x")

In [ ]:
test("x * exp(x)")

In [ ]:
test("ln(exp(x))")

As we can see, the function works as advertised.  However, in order for this approach to be really useful we would have to implement a function `simplify` that takes an arithmetic expression $e$ and then simplifies this expression.